In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
# Monta el Google Drive
drive.mount('/content/drive')

In [ ]:
base =  pd.read_csv('/content/drive/MyDrive/Seminario de Titulo - Prof Roberto Muñoz/Maximiliano -  Generación y Análisis de Secuencias de Actividad/logs/Logs-FundaProgra-2024-1.csv') #Aca se pone la ruta del logs si es CSV O EXCEL

In [ ]:
base

In [ ]:
base.info()

In [ ]:
base.describe()

In [ ]:

eventos_profesor_out= [
"Lista de usuarios vista", "Preguntas exportadas","Pregunta creada","Intento de cuestionario recalificado","Pregunta calificada manualmente",
"Rol sin asignar","Matriculación del usuario actualizada","Evento de calendario borrado","Elemento de calificación eliminado","Calificación borrada",
"Pregunta actualizada","Categoría de preguntas creada","Categoría de preguntas actualizada","Pregunta movida","Rol asignado","Usuario matriculado en el curso",
 "Categoría de preguntas borrada","Categoría de preguntas movida","Preguntas importadas","Sección de curso eliminada","Instancia de inscripcion cerrada"
]


In [ ]:
print(len(base['Nombre evento'].unique()))

In [ ]:
# --- ¡¡¡CORRECCIÓN AQUÍ!!! Limpiar 'Nombre evento' ANTES de filtrar ---
# Estandarizar la columna 'Nombre evento' para asegurar coincidencias
if 'Nombre evento' in base.columns:
    print("\nDEBUG: Limpiando columna 'Nombre evento' para el filtrado...")
    base['Nombre evento'] = base['Nombre evento'].astype(str).str.strip().str.lower()
    # Estandarizar también la lista de eventos_profesor_out a minúsculas
    eventos_profesor_out_lower = [e.lower() for e in eventos_profesor_out]
else:
    print("❌ ERROR: La columna 'Nombre evento' no existe. No se puede filtrar eventos de profesor.")
    eventos_profesor_out_lower = [] # Vacía para evitar errores si la columna no existe


In [ ]:
base = base[~base['Nombre evento'].isin(eventos_profesor_out)]





In [ ]:
print(len(base['Nombre evento'].unique()))

In [ ]:
# Verificar si las columnas existen en el DataFrame antes de eliminarlas
columnas_a_eliminar = ['Dirección IP', 'Origen', 'Usuario afectado', 'Descripción']
columnas_existentes = [col for col in columnas_a_eliminar if col in base.columns]
# Eliminar las columnas que existen
base = base.drop(columns=columnas_existentes)


In [ ]:
# Verificar las columnas restantes
print(base.columns)


In [ ]:
# Convertir la columna de fecha y hora (supongamos que se llama 'hora')
#base['Fecha'] = pd.to_datetime(base['Fecha'], format='%d/%m/%y', errors='coerce')  # Convierte la columna 'Fecha'
#base['Hora'] = pd.to_datetime(base['Hora'], format='%H:%M:%S', errors='coerce').dt.time
base['Nombre completo del usuario'] = base['Nombre completo del usuario'].astype('string')
base['Contexto del evento'] = base['Contexto del evento'].astype('string')
base['Componente'] = base['Componente'].astype('category')
base['Nombre evento'] = base['Nombre evento'].astype('category')



Nulls y duplicados

In [ ]:
#Duplicados
duplicados = base.duplicated().sum()
print(f"Número de duplicados: {duplicados}")


In [ ]:
base = base.drop_duplicates()


In [ ]:
#Duplicados
duplicados = base.duplicated().sum()
print(f"Número de duplicados: {duplicados}")

In [ ]:
base

In [ ]:
# Verificar valores duplicados tras la limpieza
faltantes = base.isnull().sum()
print(f"Datos faltantes por columna:\n{faltantes}")

In [ ]:
# Verificar valores nulos tras la limpieza
base = base.dropna()

Cambio de nombres de los alumnos

In [ ]:
base['Nombre completo del usuario'] = (
    base['Nombre completo del usuario']
    .str.strip()             # Elimina espacios al inicio y al final
    .str.replace(r'\s+', ' ', regex=True)  # Sustituye múltiples espacios por uno solo
    .str.lower()             # Convierte a minúsculas
)

In [ ]:
base

In [ ]:
# Verificar cuántos valores únicos quedan en la columna 'Nombre completo del usuario'
numero_unicos = len(base['Nombre completo del usuario'].unique())

# Mostrar el número de valores únicos
print(f"Cantidad de valores únicos en 'Nombre completo del usuario': {numero_unicos}")

Eliminar profesores, ayudantes ,etc

In [ ]:
lista_profesor = ['Roberto Muñoz','Pablo Olivares','Rodrigo Olivares','Gabriel Astudillo','Victor Rios','Daniel Diaz','Dayanna Palma','Victor Rios','Maximo Martinez','Giglia Gomez','Claudia Araya','Marta Barria','Rosa Velasquez','Yaravi Villegas','Maximo Perez','Diego Miranda','Igor Vega','Claudio Araya','Francisco Muñoz','Benjamin Serrano','Admin user','italo vergara']
lista_profesor = [nombre.lower() for nombre in lista_profesor]
lista_profesor


In [ ]:
# Función para extraer el primer nombre y primer apellido
def extract_nombre_apellido(row):
    name_parts = row.split()  # Dividir el nombre completo en partes
    #print(f'name: {name_parts}')
    # Tomar el primer nombre
    primer_nombre = name_parts[0]

    primer_apellido = name_parts[2] if len(name_parts) > 3 else (name_parts[1] if len(name_parts) > 1 else '')

    #return primer_nombre, primer_apellido


    # Concatenar el primer nombre y el primer apellido
    return f"{primer_nombre} {primer_apellido}".lower()

# Aplicar la función para crear la columna 'Nombre_apellido' en el dataframe
base['Nombre_apellido'] = base['Nombre completo del usuario'].apply(extract_nombre_apellido)


# Filtrar las filas del dataframe que NO contienen nombres de profesores
df_filtrado = base[~base['Nombre_apellido'].isin(lista_profesor)]


df_filtrado = df_filtrado.drop('Nombre_apellido', axis=1)

# Verificar el resultado
df_filtrado

In [ ]:

# Verificar cuántos valores únicos quedan en la columna 'Nombre completo del usuario'
numero_unicos2 = len(df_filtrado['Nombre completo del usuario'].unique())

# Mostrar el número de valores únicos
print(f"Cantidad de valores únicos en 'Nombre completo del usuario': {numero_unicos2}")


Habian nombres con  - , admin user que fue encontrado dsp y fueron eliminados

In [ ]:
# Filtrar filas donde 'Nombre completo del usuario' NO contiene ciertos nombres
df_filtrado = df_filtrado[
    ~df_filtrado['Nombre completo del usuario'].str.contains('admin user|italo vergara|-', case=False, na=False)
].copy()


# Verificar que se eliminaron correctamente
print(df_filtrado.shape)  # Verifica cantidad de filas y columnas después de la limpieza
print(df_filtrado['Nombre completo del usuario'].unique())  # Confirma que ya no existen nombres con 'admin user'

In [ ]:
# Ordenar por Nombre completo del usuario y Fecha
df_filtrado = df_filtrado.sort_values(by=['Nombre completo del usuario', 'Fecha'])

#Definir semana del semestre - dependiendo del semestre 1 o 2

In [ ]:
df_filtrado['Fecha'] = pd.to_datetime(df_filtrado['Fecha'], errors='coerce')

# Encontrar el primer log del semestre
primer_log = df_filtrado['Fecha'].min()

# Determinar si es el primer o segundo semestre en base al primer log
if primer_log.month <= 7:  # Primer semestre (marzo a julio)
    # Obtener el último log de julio
    ultimo_log_julio = df_filtrado[(df_filtrado['Fecha'].dt.month == 7) & (df_filtrado['Fecha'].dt.year == primer_log.year)]['Fecha'].max()
    # Definir el inicio del semestre (primer log) y el fin del semestre (último log de julio)
    inicio_semestre = primer_log
    fin_semestre = ultimo_log_julio
else:  # Segundo semestre (agosto a diciembre)
    # Obtener el último log de diciembre
    ultimo_log_agosto = df_filtrado[(df_filtrado['Fecha'].dt.month == 12) & (df_filtrado['Fecha'].dt.year == primer_log.year)]['Fecha'].max()
    # Definir el inicio del semestre (primer log) y el fin del semestre (último log de agosto)
    inicio_semestre = primer_log
    fin_semestre = ultimo_log_agosto

# Filtrar los logs entre el primer log y el final del semestre (último log de julio o agosto)
df_filtrado = df_filtrado[(df_filtrado['Fecha'] >= inicio_semestre) & (df_filtrado['Fecha'] <= fin_semestre)]

# Ahora calculamos la semana del semestre con base en el primer log
df_filtrado['semana_semestre'] = ((df_filtrado['Fecha'] - inicio_semestre).dt.days // 7) + 1

# Verificar el resultado
print(df_filtrado[['Nombre completo del usuario', 'Fecha', 'semana_semestre']].head())


Verificar que semana es la semana del segundo certamen

In [ ]:
# Contamos las interacciones de cada alumno por semana
df_filtrado['week_logs'] = df_filtrado.groupby(['Nombre completo del usuario', 'semana_semestre'])['semana_semestre'].transform('count')


#Verificar cuando es el 2 certamen

# Filtrar los logs hasta la semana 13 (antes del segundo certamen)
# Se corrige el nombre de la variable de df_max_week_logs_per_week a df_filtrado
# Se corrige semana_semestre.max() a df_filtrado['semana_semestre'].max()
df_filtered_logs = df_filtrado[df_filtrado['semana_semestre'] <= df_filtrado['semana_semestre'].max()]

# Agrupar los logs por semana para ver la cantidad total de logs (máximo por estudiante)
# Se usa df_filtered_logs que ya tiene la columna 'week_logs'
logs_per_week_filtered = df_filtered_logs.groupby('semana_semestre')['week_logs'].sum()

# Graficar los logs de la semana 1 a la semana 13

plt.figure(figsize=(10, 6))
logs_per_week_filtered.plot(kind='bar', color='lightgreen')
plt.title('Cantidad de logs por semana (Semana 1 a Semana 13)')
plt.xlabel('Semana del semestre')
plt.ylabel('Cantidad total de logs (máximo por estudiante)')
plt.xticks(rotation=45)
plt.show()

Identificado cuando fue el segundo certame, se redujo el dataset hasta la semana

In [ ]:
#Identificar semana del 2 certmaen

df_filtrado2 = df_filtrado[df_filtrado['semana_semestre'] <= 13].copy() #aca va la semana del 2 certmaen

In [ ]:
df_filtrado2.shape

In [ ]:
df_filtrado2

In [ ]:
# Guardar el DataFrame en un archivo CSV
df_filtrado2.to_csv('/content/drive/MyDrive/Seminario de Titulo - Prof Roberto Muñoz/Maximiliano -  Generación y Análisis de Secuencias de Actividad/logs/df_semestre_full_2024_S1_test.csv', index=False)

print("Archivo CSV guardado correctamente.")